# Surface restoring datasets

### Regrid surface salinity and potential temperature from land filled WOA dataset to the ocean model grid.

In [1]:
import xarray as xr
import numpy as np
from datetime import datetime
import os, subprocess
import xesmf

### Read tx1_4 grid

In [31]:
ds_out = xr.open_dataset('../mesh/tx1_4_grid.nc').rename({'tlon': 'lon','tlat': 'lat', 'qlon': 'lon_b','qlat': 'lat_b',})

### Read WOA file with land fill

In [14]:
# WOA sfc state file with land fill, created using create_filled_sfc.py
fname = '/glade/work/gmarques/cesm/mom6_input_files/WOA_MOM6/woa18_sfc_state_filled.nc'
woa = xr.open_dataset(fname, decode_times=False)

In [15]:
# average between two-layers (depth = 0 and depth = 10, depth indices 0 and 2)
woa_s_an_surface_ave = woa.s_an.isel(depth=[0,2]).mean('depth')
woa_theta0_surface_ave = woa.theta0.isel(depth=[0,2]).mean('depth')

In [22]:
def regrid_tracer(fld, ds_in, ds_out, method='bilinear'):

    regrid = xesmf.Regridder(
        ds_in,
        ds_out,
        method=method,
        periodic=True,
    )
    fld_out = regrid(fld)
    return fld_out

In [ ]:
ds_out_s_an = regrid_tracer(woa_s_an_surface_ave, ds_in, ds_out)

In [ ]:
ds_out_theta0 = regrid_tracer(woa_theta0_surface_ave, ds_in, ds_out)

### Create state file for MOM6
We opted to create this file via ncgen to avoid issues with FMS reading the netCDF file.

In [38]:
!ncgen -o state_restore_tx1_4.nc state_restore_tx1_4.cdl

In [39]:
state = xr.open_dataset('state_restore_tx1_4.nc', decode_times=False)

In [54]:
ds_out

<xarray.Dataset>
Dimensions:  (ny: 1080, nx: 1440, nxp: 1441, nyp: 1081)
Dimensions without coordinates: ny, nx, nxp, nyp
Data variables: (12/20)
    lon      (ny, nx) float64 -286.9 -286.6 -286.4 -286.1 ... 73.0 73.0 73.0
    lat      (ny, nx) float64 -80.03 -80.03 -80.03 -80.03 ... 50.03 50.0 50.0
    ulon     (ny, nxp) float64 -287.0 -286.8 -286.5 -286.2 ... 73.0 73.0 73.0
    ulat     (ny, nxp) float64 -80.03 -80.03 -80.03 -80.03 ... 50.01 50.0 50.0
    vlon     (nyp, nx) float64 -286.9 -286.6 -286.4 -286.1 ... 73.0 73.0 73.0
    vlat     (nyp, nx) float64 -80.05 -80.05 -80.05 -80.05 ... 50.03 50.0 50.0
    ...       ...
    tarea    (ny, nx) float64 2.315e+07 2.315e+07 2.315e+07 ... 3.339e+05 198.9
    tmask    (ny, nx) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    angle    (ny, nx) float64 -0.06156 -0.06156 -0.06156 ... -4.024 -0.4197
    depth    (ny, nx) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    ar       (ny, nx) float64 1.0 1.0 1.0 1.0 ... 0.1318 0.1046 0.281 1.457e+03
    egs      (ny, nx) float64 0.0004327 0.0004327 ... 5.196e-05 1.268e-06
Attributes:
    Description:  CESM MOM6 1/4 degree grid
    Author:       Frank, Fred, Gustavo (gmarques@ucar.edu)
    Created:      2022-12-27T10:20:11.076318
    type:         Glogal 1/4 degree grid file

In [60]:
jm, im = ds_out.lat.shape
state['LAT'] = ds_out.lat[:,int(im/2)].values
state['LON'] = ds_out.lon[int(jm/2),:].values

Overwrite salt and thetao

In [44]:
state['salt'].values[:] = ds_out_s_an.values[:]

In [47]:
state['theta0'].values[:] = ds_out_theta0.values[:]

TODO: Compare original and remapped fields

In [66]:
# Global attrs
state.attrs['title'] = 'surface salinity and potential temperature from WOA filled over continents'
state.attrs['src_file'] = fname
state.attrs['dst_grid_name'] = 'tx1_4'
state.attrs['author'] = 'Gustavo Marques (gmarques@ucar.edu)'
state.attrs['date'] = datetime.now().isoformat()
state.attrs['created_using'] = 'https://github.com/NCAR/tx1_4/state_restoring/state_restoring.ipynb'
# save 20200616
fname1 = 'state_restore_tx1_4_{}{}{}.nc'.format(datetime.now().isoformat()[0:4],datetime.now().isoformat()[5:7],
       datetime.now().isoformat()[8:10])
state.to_netcdf(fname1)